In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from fastapi.responses import JSONResponse
from typing import List, Literal
from contextlib import asynccontextmanager
import uvicorn
   
RATING_MAP = {
    'Not Interested': 0,
    'Beginner': 1,
    'Average': 2,
    'Excellent': 3
}

# ==============================================
# Training and saving models
# ==============================================

def train_and_save_models():
    # Load data
    df = pd.read_csv("transformed_dataset.csv")
    
 
    for col in df.columns:
        if col != 'Track' and df[col].dtype == object:
            df[col] = df[col].map(RATING_MAP)
    
    # Ensure all values are between 0 and 3
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].clip(0, 3)
    
    # Encode target
    le = LabelEncoder()
    y = le.fit_transform(df['Track'])
    
    # Get features by dropping target column
    X = df.drop('Track', axis=1)
    feature_names = X.columns.tolist()
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, stratify=y, random_state=42
    )
    
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Dictionary to store models
    models = {
        'XGBoost': XGBClassifier(
            max_depth=5,
            learning_rate=0.1,
            n_estimators=200,
            objective='multi:softmax',
            num_class=len(le.classes_),
            eval_metric='merror'
        ),
        'RandomForest': RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            random_state=42
        ),
        'SVM': SVC(
            C=1.0,
            kernel='rbf',
            probability=True,
            random_state=42
        ),
        'KNN': KNeighborsClassifier(
            n_neighbors=5
        )
    }
    
    # Train and evaluate each model
    best_model = None
    best_accuracy = 0
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = model.predict(X_test_scaled)
        y_test_decoded = le.inverse_transform(y_test)
        y_pred_decoded = le.inverse_transform(y_pred)
        
        print(classification_report(y_test_decoded, y_pred_decoded))
        train_acc = model.score(X_train_scaled, y_train)
        test_acc = model.score(X_test_scaled, y_test)
        print(f"Train Accuracy: {train_acc:.2%}")
        print(f"Test Accuracy: {test_acc:.2%}")
        
        # Track best model
        if test_acc > best_accuracy:
            best_accuracy = test_acc
            best_model = model
    
    # Save the best model and tools
    if not os.path.exists("Models"):
        os.makedirs("Models")
    
    joblib.dump({
        'model': best_model,
        'scaler': scaler,
        'label_encoder': le,
        'feature_names': feature_names,
        'best_model_name': type(best_model).__name__,
        'rating_map': RATING_MAP
    }, "Models/best_model.pkl")
    
    print(f"\n✅ Best model saved successfully! Model: {type(best_model).__name__}")

# ==============================================
# FastAPI part
# ==============================================

# Global variables
model = None
scaler = None
le = None
feature_names = None
model_name = None
rating_map = None

# Lifespan management
@asynccontextmanager
async def lifespan(app: FastAPI):
    global model, scaler, le, feature_names, model_name, rating_map
    try:
        model_data = joblib.load("Models/best_model.pkl")
        model = model_data['model']
        scaler = model_data['scaler']
        le = model_data['label_encoder']
        feature_names = model_data['feature_names']
        model_name = model_data['best_model_name']
        rating_map = model_data['rating_map']
        print(f"✅ Models loaded successfully. Best model: {model_name}")
    except Exception as e:
        print(f"❌ Failed to load models: {str(e)}")
        raise
    yield

app = FastAPI(
    title="Technical Tracks Recommendation System",
    description="Recommends top 3 technical tracks based on user skill levels",
    version="1.0",
    lifespan=lifespan
)

# Define possible skill levels
SkillLevel = Literal['Not Interested', 'Beginner', 'Average', 'Excellent']

# User input model with text skill levels
class UserInput(BaseModel):
    Programming_Skills: SkillLevel = Field('Beginner', example='Beginner')
    Computer_Architecture: SkillLevel = Field('Beginner', example='Beginner')
    Technical_Communication: SkillLevel = Field('Beginner', example='Beginner')
    Cyber_Security: SkillLevel = Field('Beginner', example='Beginner')
    AI: SkillLevel = Field('Beginner', example='Beginner')
    Biology_Biochemistry_Basic: SkillLevel = Field('Beginner', example='Beginner')
    Networking: SkillLevel = Field('Beginner', example='Beginner')
    Troubleshooting_Skills: SkillLevel = Field('Beginner', example='Beginner')
    Database_Fundamentals: SkillLevel = Field('Beginner', example='Beginner')
    Leadership_Experience: SkillLevel = Field('Beginner', example='Beginner')
    Project_Management: SkillLevel = Field('Beginner', example='Beginner')
    Software_Engineering: SkillLevel = Field('Beginner', example='Beginner')
    Business_knowledge: SkillLevel = Field('Beginner', example='Beginner')
    Communication_Skills: SkillLevel = Field('Beginner', example='Beginner')
    Data_Science: SkillLevel = Field('Beginner', example='Beginner')
    Graphics_Designing: SkillLevel = Field('Beginner', example='Beginner')

# Output model (simplified to show only track names)
class Recommendation(BaseModel):
    track: str

class RecommendationsResponse(BaseModel):
    top_recommendations: List[Recommendation]
    message: str = "Recommendations generated successfully"
    model_used: str

@app.get("/")
async def root():
    return {"message": "Welcome to the Technical Tracks Recommendation System"}

@app.post("/recommend", response_model=RecommendationsResponse)
async def get_recommendations(user_input: UserInput):
    try:
        # Convert input to dictionary and then map to numerical values
        input_data = user_input.model_dump()
        numerical_data = {k: rating_map[v] for k, v in input_data.items()}
        
        # Create DataFrame
        input_df = pd.DataFrame([numerical_data])[feature_names]
        
        # Preprocess
        X_scaled = scaler.transform(input_df)
        
        # Predict probabilities
        probs = model.predict_proba(X_scaled)[0]
        
        # Get top 3 recommendations
        top3_indices = np.argsort(probs)[-3:][::-1]
        top3_tracks = le.inverse_transform(top3_indices)
        
        # Prepare response 
        recommendations = [{"track": track} for track in top3_tracks]
        
        return {
            "top_recommendations": recommendations,
            "message": "Recommendations generated successfully",
            "model_used": model_name
        }
    
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error processing your request: {str(e)}"
        )

@app.get("/tracks")
async def get_available_tracks():
    return {
        "available_tracks": le.classes_.tolist(),
        "count": len(le.classes_)
    }

# ==============================================
# Main execution
# ==============================================

if __name__ == "__main__":
    # Train model first
    print("Training the model...")
    train_and_save_models()
    
    # Run server
    import nest_asyncio
    nest_asyncio.apply()
    
    uvicorn.run(app, host="127.0.0.1", port=8000)

Training the model...

Training XGBoost...
                          precision    recall  f1-score   support

     Backend Development       0.95      0.94      0.95       150
  Blockchain Development       0.96      0.81      0.88       180
      Cloud Architecture       0.69      0.93      0.79       180
       Computer Literacy       0.56      0.87      0.68       210
           Cybersecurity       0.94      0.91      0.93       150
           Data Analysis       0.93      0.91      0.92       150
 Database Administration       0.93      0.94      0.93       150
      DevOps Engineering       0.92      0.73      0.81       180
    Digital Productivity       0.87      0.70      0.78       180
    Frontend Development       0.97      0.98      0.97       150
         IT Fundamentals       0.84      0.65      0.73       180
   IT Project Management       0.96      0.78      0.86       180
        Machine Learning       0.96      0.96      0.96       150
    Network Fundamentals       0

INFO:     Started server process [28664]


Train Accuracy: 81.08%
Test Accuracy: 74.98%

✅ Best model saved successfully! Model: XGBClassifier


INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


✅ Models loaded successfully. Best model: XGBClassifier
INFO:     127.0.0.1:63920 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63920 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:63921 - "POST /recommend HTTP/1.1" 200 OK
